In [1]:
!nvidia-smi

Fri Nov 22 02:39:30 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    31W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from tensorflow.keras import backend as K
K.clear_session()

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
import tensorflow as tf
import numpy as np
import wandb
import time

In [4]:
wandb.init(project='mixed-precision-tf-keras', name='with-mp_loss_scale-128')

wandb: Wandb version 0.8.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


W&B Run: https://app.wandb.ai/sayakpaul/mixed-precision-tf-keras/runs/9xca54na

In [5]:
# Enable XLA
tf.config.optimizer.set_jit(True)

# Enable AMP
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

In [6]:
DATA_PATH = '/home/jupyter/Mixed-Precision-Training-in-tf.keras-2.0/Data'

X_train = np.load(DATA_PATH + '/X_train.npy', allow_pickle=True)
y_train = np.load(DATA_PATH + '/y_train.npy', allow_pickle=True)

In [7]:
print(X_train.shape, y_train.shape)
print(X_train.dtype)

(6252, 224, 224, 3) (6252,)
float64


In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [9]:
IMG_SIZE = (128, 128)

@tf.function
def format_train_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, IMG_SIZE)
    return image, tf.one_hot(label, len(np.unique(y_train)))

In [10]:
BATCH_SIZE = 80
AUTOTUNE = tf.data.experimental.AUTOTUNE
PREFETCH_COUNT = 8

train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.map(format_train_example,
                                  num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(PREFETCH_COUNT)

In [11]:
def create_model(img_size=(128,128), num_class=5, train_base=True):
    input_layer = Input(shape=(img_size[0],img_size[1],3))
    base = ResNet50(input_tensor=input_layer,
                    include_top=False,
                    weights="imagenet")
    base.trainable = train_base
    x = base.output
    x = GlobalAveragePooling2D()(x)
    preds = Dense(num_class, activation="softmax")(x)
    return Model(inputs=input_layer, outputs=preds)

In [12]:
model = create_model()
opt = Adam()
opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt,  
                                                       "dynamic")
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [13]:
# Kickstart model training
start = time.time()
model.fit(train_dataset, 
         steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
         epochs=5,
         callbacks=[WandbCallback()])
print("It took {} seconds".format(time.time() - start))

wandb: Wandb version 0.8.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Train for 78 steps
Epoch 1/5
78/78 [==============================] - 61s 786ms/step - loss: 0.7259 - accuracy: 0.5840
Epoch 2/5
78/78 [==============================] - 7s 96ms/step - loss: 0.8616 - accuracy: 0.4893
Epoch 3/5
78/78 [==============================] - 8s 96ms/step - loss: 1.0468 - accuracy: 0.3958
Epoch 4/5
78/78 [==============================] - 8s 97ms/step - loss: 1.1263 - accuracy: 0.3567
Epoch 5/5
78/78 [==============================] - 8s 98ms/step - loss: 1.1096 - accuracy: 0.3713
It took 94.78035426139832 seconds
